# 目前对fmri图像的理解
此处仅导入接下来演示所需要的库

In [2]:
import numpy as np
import nilearn
from nilearn.input_data import NiftiMasker
import nibabel as nib
import matplotlib.pyplot as plt

import os
import random
import math
import time

C:\Users\86189\AppData\Roaming\Python\Python36\site-packages\nilearn\datasets\__init__.py:96: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


## ADHD-200数据集
我们计划使用ADHD-200数据集作为今后开展研究课题的数据。（并不一定也有可能会使用HCP900）<br>
ADHD-200是一个多动症病人的数据集，里面的图像内容包含四种不同的分类，分为以下四类(中文为机翻，具体是否如此有待考察)：<br>
Typically-Developing Controls 典型开发控件<br>
ADHD Combined 多动症结合<br>
ADHD Inattentive 注意力不集中的多动症<br>
ADHD Hyperactive 多动症多动症<br>
每一张图像都有对应的dmri，和fmri的图像，其中dmri只有三个维度，也就是只有$（x，y，z）$三个坐标方向。fmri则加上了时间维度，一共有四个维度$（x，y，z，t）$

### 为什么选择ADHD-200
- 相对于HCP900，ADHD-200数据集更小，下载使用更为方便，完整的ADHD200数据集仅82G，而完整的HCP900以TB计算。
- ADHD数据集有明确的目标，对多动症病人进行分类，分类任务是一个较容易理解的任务。

### ADHD-200数据集下载
ADHD-200数据集下载方式有两种，一种是完整的在官网下载所有数据集[下载链接](https://www.nitrc.org/ir/app/template/XDATScreen_report_xnat_projectData.vm/search_element/xnat:projectData/search_field/xnat:projectData.ID/search_value/adhd_200)。另一种是通过nilearn下载，但是通过nilearn下载的数据是不完整的，最多只能包含40个图像，以下是如何通过nilearn下载数据。<br>
```
#将会下载1G数据，而且网不好容易断掉，此处不做演示
from nilearn import datasets
adhd_dataset = datasets.fetch_adhd(n_subjects=40, "在此填写下载到具体哪个位置")
```

通过nilearn下载的数据和在官网下载的数据并不一样，经过我的观察体现在nilearn下载的数据是经过处理的图像，所有图像统一了size大小和时间步长短。

In [15]:
img = nib.load("../stAAE/data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz")
img.shape #nilearn下载的图像size

(61, 73, 61, 176)

此数据集包含Brown，KKI，NeuroIMAGE，NYU，OHSU，Peking，Pittsburgh，WashU多个不同组织上传的数据，其中每个组织上传的数据尺寸都各不相同。表现的颇为混乱。以下代码输出：左为图像size和时间步，右为这样形状的图像个数。

In [4]:
d = {}
root = "../../ADHD/whataboutmystar-20220130_215152/"
dir_names = os.listdir(root)

#此处是一个深度优先搜索，用以遍历目录中所有nii.gz的mri图像
def DFS(path):
    if os.path.isdir(path):
        dir_names = os.listdir(path)
        for name in dir_names:
            DFS(path + '/' + name)
    elif path.endswith(".gz"):
        data = nib.load(path)
        d[data.shape] = d.get(data.shape, 0) + 1
    else:
        return
    
    return

for name in dir_names:
    DFS(root + name)
d

{(160, 240, 256): 147,
 (64, 64, 35, 251): 26,
 (256, 200, 256): 86,
 (96, 96, 47, 152): 28,
 (96, 96, 47, 124): 54,
 (256, 180, 256): 7,
 (112, 112, 47, 152): 1,
 (96, 96, 47, 123): 3,
 (96, 96, 47, 151): 7,
 (96, 96, 47, 157): 1,
 (256, 190, 256): 1,
 (176, 256, 256): 132,
 (64, 64, 37, 261): 63,
 (64, 64, 35, 261): 5,
 (64, 64, 37, 260): 5,
 (128, 256, 256): 400,
 (64, 80, 33, 176): 396,
 (64, 80, 33, 175): 41,
 (64, 64, 36, 78): 233,
 (64, 64, 36, 76): 3,
 (64, 64, 36, 77): 34,
 (46, 240, 256): 1,
 (64, 64, 36, 59): 1,
 (64, 64, 33, 235): 51,
 (176, 512, 512): 20,
 (64, 64, 30, 236): 42,
 (176, 208, 256): 67,
 (64, 64, 33, 236): 152,
 (192, 256, 256): 20,
 (128, 240, 256): 1,
 (208, 256, 176): 89,
 (64, 64, 29, 196): 89,
 (64, 64, 46, 123): 9,
 (64, 64, 32, 76): 24,
 (64, 64, 32, 133): 35,
 (64, 64, 32, 132): 112,
 (256, 256, 256): 2}

由上输出可知无论是时间步还是图像size都不一样，需要经过处理才能使用。具体说来是把图像size处理成同样大小，图像时间步也处理成同样大小。

## fmri图像本质探究

已知现实生活中的彩色图像也就是RGB图像是二维图像，由三个颜色通道叠加在一起组成，每个颜色通道都是一个二维网格。每个网格上都有一个像素点，像素大小只能是0-255。<br>
但是核磁共振图像则不一样，核磁共振是三维的图像，每个图像由一个三维网格组成，每个网格就是一个小正方体，每个正方体就是一个体素，每个体素和像素的概念是对应的，包含有一个具体的值，只不过像素的值只能是0-255（灰度图除外），fmri的体素应该是代表信号的强度，大脑的活跃程度。数值不知道最高能多少，我最高见过高达40000多的值。最低的值也有达到负数的。
![voxel](./img/voxel.png)

In [16]:
data = img.get_fdata()
data.min()

0.0

In [18]:
data.max()

24359.931640625